# 25 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [3]:
no_of_patients=25

In [4]:
train=sorted(glob('files4/*.edf'))
train=train[:no_of_patients]
train

['files4/S001R06.edf',
 'files4/S002R06.edf',
 'files4/S003R06.edf',
 'files4/S004R06.edf',
 'files4/S005R06.edf',
 'files4/S006R06.edf',
 'files4/S007R06.edf',
 'files4/S008R06.edf',
 'files4/S009R06.edf',
 'files4/S010R06.edf',
 'files4/S011R06.edf',
 'files4/S012R06.edf',
 'files4/S013R06.edf',
 'files4/S014R06.edf',
 'files4/S015R06.edf',
 'files4/S016R06.edf',
 'files4/S017R06.edf',
 'files4/S018R06.edf',
 'files4/S019R06.edf',
 'files4/S020R06.edf',
 'files4/S021R06.edf',
 'files4/S022R06.edf',
 'files4/S023R06.edf',
 'files4/S024R06.edf',
 'files4/S025R06.edf']

In [5]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [7]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [8]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [9]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(300000, 37475, 300000, 37475)

In [10]:
print(xtest.iloc[:,-1].values)

[-5.8e-05 -3.7e-05 -1.0e-06 ... -2.0e-06 -6.0e-06  1.0e-05]


In [11]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.000003,-0.000003,0.000012,-0.000006,-0.000007,0.000010,-0.000013,-0.000012,-0.000010,-0.000003,...,-0.000003,-0.000005,-0.000009,-0.000011,-0.000017,-0.000011,-0.000010,-0.000021,0.000000,-0.000011
299996,0.000019,0.000000,0.000009,-0.000010,-0.000016,-0.000002,-0.000020,-0.000016,-0.000014,-0.000010,...,-0.000010,0.000001,-0.000007,-0.000012,-0.000019,-0.000012,-0.000007,-0.000018,-0.000002,-0.000007
299997,0.000008,-0.000007,0.000001,-0.000019,-0.000026,-0.000012,-0.000029,-0.000014,-0.000017,-0.000012,...,-0.000001,-0.000002,-0.000002,0.000000,-0.000006,-0.000001,-0.000002,-0.000012,0.000007,-0.000004
299998,0.000000,-0.000005,0.000009,-0.000011,-0.000015,-0.000002,-0.000019,-0.000013,-0.000013,-0.000002,...,-0.000001,-0.000006,-0.000002,0.000005,0.000006,0.000018,-0.000004,-0.000007,0.000021,-0.000006


In [12]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [13]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [14]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_62567/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_62567/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_62567/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.000003,-0.000003,0.000012,-0.000006,-0.000007,0.000010,-0.000013,-0.000012,-0.000010,-0.000003,...,-0.000003,-0.000005,-0.000009,-0.000011,-0.000017,-0.000011,-0.000010,-0.000021,0.000000,-0.000011
299996,0.000019,0.000000,0.000009,-0.000010,-0.000016,-0.000002,-0.000020,-0.000016,-0.000014,-0.000010,...,-0.000010,0.000001,-0.000007,-0.000012,-0.000019,-0.000012,-0.000007,-0.000018,-0.000002,-0.000007
299997,0.000008,-0.000007,0.000001,-0.000019,-0.000026,-0.000012,-0.000029,-0.000014,-0.000017,-0.000012,...,-0.000001,-0.000002,-0.000002,0.000000,-0.000006,-0.000001,-0.000002,-0.000012,0.000007,-0.000004
299998,0.000000,-0.000005,0.000009,-0.000011,-0.000015,-0.000002,-0.000019,-0.000013,-0.000013,-0.000002,...,-0.000001,-0.000006,-0.000002,0.000005,0.000006,0.000018,-0.000004,-0.000007,0.000021,-0.000006


In [15]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
9375/9375 [==============================] - 6s 665us/step - loss: 1.7511 - val_loss: 2.6036
Epoch 2/10
9375/9375 [==============================] - 6s 661us/step - loss: 1.2737 - val_loss: 2.5230
Epoch 3/10
9375/9375 [==============================] - 6s 657us/step - loss: 1.1525 - val_loss: 2.3763
Epoch 4/10
9375/9375 [==============================] - 6s 673us/step - loss: 1.1001 - val_loss: 2.5393
Epoch 5/10
9375/9375 [==============================] - 6s 685us/step - loss: 1.0700 - val_loss: 2.5290
Epoch 6/10
9375/9375 [==============================] - 6s 675us/step - loss: 1.0468 - val_loss: 2.5569
Epoch 7/10
9375/9375 [==============================] - 6s 672us/step - loss: 1.0314 - val_loss: 2.6655
Epoch 8/10
9375/9375 [==============================] - 6s 679us/step - loss: 1.0170 - val_loss: 2.6334
Epoch 9/10
9375/9375 [==============================] - 6s 667us/step - loss: 1.0072 - val_loss: 2.8907
Epoch 10/10
9375/9375 [==============================] - 6s 684u

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

1172/1172 [==============================] - 0s 290us/step
              precision    recall  f1-score   support

           0       0.28      0.28      0.28      1499
           1       0.15      0.19      0.17      1499
           2       0.47      0.34      0.39      1499
           3       0.67      0.72      0.70      1499
           4       0.81      0.73      0.77      1499
           5       0.70      0.67      0.68      1499
           6       0.60      0.61      0.60      1499
           7       0.55      0.72      0.62      1499
           8       0.70      0.71      0.71      1499
           9       0.61      0.38      0.47      1499
          10       0.51      0.27      0.35      1499
          11       0.34      0.47      0.40      1499
          12       0.31      0.24      0.27      1499
          13       0.21      0.22      0.21      1499
          14       0.21      0.30      0.25      1499
          15       0.51      0.54      0.52      1499
          16       0.4

## 0-16

In [19]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [20]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_62567/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_62567/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_62567/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.000003,-0.000003,0.000012,-0.000006,-0.000007,0.000010,-0.000013,-0.000012,-0.000010,-0.000003,...,-0.000003,-0.000005,-0.000009,-0.000011,-0.000017,-0.000011,-0.000010,-0.000021,0.000000,-0.000011
299996,0.000019,0.000000,0.000009,-0.000010,-0.000016,-0.000002,-0.000020,-0.000016,-0.000014,-0.000010,...,-0.000010,0.000001,-0.000007,-0.000012,-0.000019,-0.000012,-0.000007,-0.000018,-0.000002,-0.000007
299997,0.000008,-0.000007,0.000001,-0.000019,-0.000026,-0.000012,-0.000029,-0.000014,-0.000017,-0.000012,...,-0.000001,-0.000002,-0.000002,0.000000,-0.000006,-0.000001,-0.000002,-0.000012,0.000007,-0.000004
299998,0.000000,-0.000005,0.000009,-0.000011,-0.000015,-0.000002,-0.000019,-0.000013,-0.000013,-0.000002,...,-0.000001,-0.000006,-0.000002,0.000005,0.000006,0.000018,-0.000004,-0.000007,0.000021,-0.000006


In [21]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
9375/9375 [==============================] - 7s 701us/step - loss: 1.2932 - val_loss: 2.2410
Epoch 2/10
9375/9375 [==============================] - 6s 677us/step - loss: 0.7348 - val_loss: 2.4089
Epoch 3/10
9375/9375 [==============================] - 6s 660us/step - loss: 0.5810 - val_loss: 2.6637
Epoch 4/10
9375/9375 [==============================] - 6s 651us/step - loss: 0.5159 - val_loss: 2.6524
Epoch 5/10
9375/9375 [==============================] - 6s 692us/step - loss: 0.4799 - val_loss: 2.7444
Epoch 6/10
9375/9375 [==============================] - 6s 691us/step - loss: 0.4568 - val_loss: 2.7384
Epoch 7/10
9375/9375 [==============================] - 6s 682us/step - loss: 0.4393 - val_loss: 3.0796
Epoch 8/10
9375/9375 [==============================] - 6s 693us/step - loss: 0.4246 - val_loss: 2.8056
Epoch 9/10
9375/9375 [==============================] - 6s 684us/step - loss: 0.4135 - val_loss: 2.8785
Epoch 10/10
9375/9375 [==============================] - 7s 697u

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

1172/1172 [==============================] - 0s 285us/step
              precision    recall  f1-score   support

           0       0.48      0.41      0.44      1499
           1       0.16      0.23      0.19      1499
           2       0.53      0.50      0.52      1499
           3       0.73      0.45      0.56      1499
           4       0.91      0.81      0.86      1499
           5       0.78      0.73      0.75      1499
           6       0.89      0.41      0.57      1499
           7       0.68      0.59      0.63      1499
           8       0.59      0.79      0.68      1499
           9       0.79      0.62      0.69      1499
          10       0.56      0.11      0.19      1499
          11       0.70      0.55      0.62      1499
          12       0.43      0.60      0.50      1499
          13       0.24      0.42      0.31      1499
          14       0.26      0.38      0.31      1499
          15       0.76      0.39      0.51      1499
          16       0.5

## 0-32

In [25]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [26]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_62567/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_62567/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_62567/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [27]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
9375/9375 [==============================] - 7s 725us/step - loss: 0.7932 - val_loss: 1.8528
Epoch 2/10
9375/9375 [==============================] - 7s 715us/step - loss: 0.2628 - val_loss: 1.7551
Epoch 3/10
9375/9375 [==============================] - 7s 707us/step - loss: 0.1860 - val_loss: 1.9163
Epoch 4/10
9375/9375 [==============================] - 7s 714us/step - loss: 0.1535 - val_loss: 1.8395
Epoch 5/10
9375/9375 [==============================] - 7s 710us/step - loss: 0.1339 - val_loss: 1.8014
Epoch 6/10
9375/9375 [==============================] - 7s 715us/step - loss: 0.1178 - val_loss: 1.6788
Epoch 7/10
9375/9375 [==============================] - 7s 720us/step - loss: 0.1066 - val_loss: 1.8251
Epoch 8/10
9375/9375 [==============================] - 7s 703us/step - loss: 0.1007 - val_loss: 1.8005
Epoch 9/10
9375/9375 [==============================] - 7s 724us/step - loss: 0.0931 - val_loss: 1.8089
Epoch 10/10
9375/9375 [==============================] - 7s 713u

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

1172/1172 [==============================] - 0s 291us/step
              precision    recall  f1-score   support

           0       0.77      0.74      0.75      1499
           1       0.34      0.16      0.22      1499
           2       0.85      0.81      0.83      1499
           3       0.52      0.34      0.41      1499
           4       0.96      0.91      0.93      1499
           5       0.98      0.83      0.90      1499
           6       0.99      0.72      0.84      1499
           7       0.77      0.52      0.62      1499
           8       0.74      0.98      0.85      1499
           9       0.91      0.91      0.91      1499
          10       0.78      0.84      0.81      1499
          11       0.97      0.85      0.91      1499
          12       0.73      0.92      0.81      1499
          13       0.42      0.92      0.58      1499
          14       0.50      0.81      0.62      1499
          15       0.88      0.59      0.71      1499
          16       0.9

## 0-64

In [31]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [32]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
9375/9375 [==============================] - 7s 778us/step - loss: 0.5132 - val_loss: 1.5659
Epoch 2/10
9375/9375 [==============================] - 7s 794us/step - loss: 0.1567 - val_loss: 1.3371
Epoch 3/10
9375/9375 [==============================] - 7s 775us/step - loss: 0.1113 - val_loss: 1.3264
Epoch 4/10
9375/9375 [==============================] - 7s 777us/step - loss: 0.0915 - val_loss: 1.2577
Epoch 5/10
9375/9375 [==============================] - 7s 766us/step - loss: 0.0775 - val_loss: 1.2057
Epoch 6/10
9375/9375 [==============================] - 7s 775us/step - loss: 0.0695 - val_loss: 1.0995
Epoch 7/10
9375/9375 [==============================] - 7s 785us/step - loss: 0.0620 - val_loss: 1.3081
Epoch 8/10
9375/9375 [==============================] - 7s 766us/step - loss: 0.0574 - val_loss: 1.1754
Epoch 9/10
9375/9375 [==============================] - 7s 774us/step - loss: 0.0539 - val_loss: 1.1682
Epoch 10/10
9375/9375 [==============================] - 7s 772u

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

1172/1172 [==============================] - 0s 308us/step
              precision    recall  f1-score   support

           0       0.88      0.60      0.71      1499
           1       0.58      0.46      0.51      1499
           2       0.85      0.86      0.86      1499
           3       0.85      0.82      0.84      1499
           4       1.00      0.99      1.00      1499
           5       0.94      0.95      0.94      1499
           6       0.94      0.89      0.91      1499
           7       0.86      0.60      0.71      1499
           8       0.84      0.98      0.91      1499
           9       0.97      0.97      0.97      1499
          10       0.83      0.48      0.61      1499
          11       0.81      0.99      0.89      1499
          12       0.83      0.91      0.87      1499
          13       0.61      0.69      0.65      1499
          14       0.60      0.89      0.72      1499
          15       0.95      0.89      0.92      1499
          16       0.9